In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Imposta l'ID della GPU che vuoi usare

import torch
import numpy as np
import json




import pdfplumber
from tqdm import tqdm
from typing import List, Optional
from pydantic import BaseModel, Field
from llama_index.core.output_parsers import PydanticOutputParser
from llama_parse import LlamaParse
import transformers
import re
from IPython.display import display, Markdown
import PyPDF2
from haystack import Document
from haystack.components.preprocessors import DocumentSplitter
from sentence_transformers import SentenceTransformer

# Scarica i dati necessari per la tokenizzazione delle frasi

# Inizializza il modello specificato
model_rag = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device="cuda")

2024-12-01 15:31:33.036762: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 15:31:33.047930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 15:31:33.061594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 15:31:33.065448: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 15:31:33.076765: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def generate_embeddings(entry, model=model_rag):
    # Generazione dell'embedding
    embedding = model.encode(entry, convert_to_numpy=True, normalize_embeddings=True)
    return embedding

In [3]:
import weaviate
import locale
from datetime import datetime

# Imposta la lingua italiana
locale.setlocale(locale.LC_TIME, 'it_IT.UTF-8')
key = 'NspY2XG4AgqNdA25wiP9jqyFChmrOXwh8tvI'
url = "https://atrawoytqmxwlvpuixika.c0.europe-west3.gcp.weaviate.cloud"

# client Weaviate
client = weaviate.Client(
    url=url,  
    auth_client_secret=weaviate.auth.AuthApiKey(api_key=key), 
)


/home/a.alessandrelli/.local/lib/python3.10/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [4]:
# rimozione di alcune stringhe di fine e inizio pagina
def cleaning(text):
    #text=re.sub(r'\s+',' ',text)
    text=re.sub(r'\n+','\n',text)
    text=re.sub(r'Pagina \d+ di \d+|pagina \d+ di \d+|\d+ di \d+','\n',text)
    text=text.replace('Immagine di Archivio','')
    text=text.replace('immagine generica illustrativa','')
    text=text.replace('Image not found or type unknown','')
    text=text.replace('I Imm agm\ne\na nog\nt\ni fn oue\nn\ndd oi\nr\nA tyr pc\ne\nh univ kni oo\nwn','')
    text=text.replace('logo FiscoOggi','')
    if 'url:' in text.lower():
        text=''

    return text

In [9]:
# pdf
import os
import pdfplumber

DATA_DIR = ["/home/a.alessandrelli/LLM/Hackaton_LLama_ROMA/DATI_DEFINITIVI/diritto-e-fisco","/home/a.alessandrelli/LLM/Hackaton_LLama_ROMA/DATI_DEFINITIVI/pdf","/home/a.alessandrelli/LLM/Hackaton_LLama_ROMA/DATI_DEFINITIVI/90k 100k", "/home/a.alessandrelli/LLM/Hackaton_LLama_ROMA/DATI_DEFINITIVI/pdf1"]
LOG_FILE = "error_log.txt"

import os

In [ ]:
files = []
for idx, c in enumerate(DATA_DIR):
    for f in os.listdir(c):
        files.append({f[:-4].strip():c+'/'+f})

In [11]:
len(files)

1280

In [12]:
def get_data_files(data_dir) -> list[dict]:
    """
    Ottiene i percorsi e i nomi dei file PDF nella directory specificata.

    :param data_dir: Directory contenente i file PDF.
    :return: Lista di dizionari con {"ID": "filepath+filename"}.
    """
    files = []
    for f in os.listdir(data_dir):
        fname = os.path.join(data_dir, f)
        if os.path.isfile(fname) and fname.endswith(".pdf") and not f.startswith("._"):
            # Aggiungi solo i file PDF che non iniziano con "._"
            files.append({"ID": fname})
    return files

In [13]:
def convertitore_data(data_testo):
    formati = ["%d %b %Y", "%d %B %Y", "%d/%m/%Y", "%Y-%m-%d", "%d-%m-%Y"]

    data_convertita = None
    for formato in formati:
        try:
            data_convertita = datetime.strptime(data_testo, formato)
            break
        except ValueError:
            pass
    if data_convertita is None:
        data_convertita = ''
    return data_convertita

In [14]:
def estrai_testo_pdfplumber(file_path1):
    file_path = list(file_path1.values())[0]
    controllo = -1
    
    testo_completo = ""
    with pdfplumber.open(file_path) as pdf:
        for pagina in pdf.pages:
            try:
                testo = pagina.extract_text()
                if testo:  # Controlla se la pagina contiene testo
                    testo_completo += testo + "\n"
            except:
                continue
    if not testo_completo.strip():  # Se non c'è testo estraibile
        raise ValueError(f"Nessun testo estraibile in {file_path}")
    text_temp = testo_completo.split('\n')
    text_clean = [cleaning(t.strip()) for t in text_temp if cleaning(t.strip()).strip()!='']
    text_clean = [t for t in text_clean if 'Articolo pubblicato su FiscoOggi' not in t]
    for pos, i in enumerate(text_clean):
        if 'scritto da' in i.replace('\n',''):
            break
    if pos < len(text_clean)/2:
        titolo = '\n'.join(text_clean[:pos])
        text_clean1 ='\n'.join(text_clean[pos+1:])
        argomento = ' '
    else:
        titolo = ' '
        text_clean1 ='\n'.join(text_clean)
        argomento = ' '
    data = ' '
    return  argomento, titolo, data, text_clean1.replace('I Imm agm\ne\na nog\nt\ni fn oue\nn\ndd oi\nr\nA tyr pc\ne\nh univ kni oo\nwn\n','').strip(), titolo+'.pdf',controllo

In [15]:
saving_path ='/home/a.alessandrelli/LLM/Hackaton_LLama_ROMA/dati_greco.jsonl'
raw_docs_long=[]
args=[]
idx=0
for f in tqdm(files[idx:]):
    try:
        temp = estrai_testo_pdfplumber(f)
    except:
        idx+=1
        continue
    argomento= temp[0]
    title= temp[1]
    data_1=temp[2]
    testo=temp[3]
    titolo_sporco = temp[4]
    if temp[-1]==100:
        continue
    else:
        raw_docs_long.append(Document(content=testo, meta={"testo_completo":testo, "title": title,"argomento":argomento,"data":str(data_1), "fonte":titolo_sporco}))
        args.append({'text':testo, "title": title,"argomento":argomento,"data":str(data_1), "fonte":titolo_sporco})
    if idx == 0:
        mode ='w'
    else:
        mode ='a'
    with open(saving_path,mode, encoding='utf-8') as file:
        file.write(json.dumps(args[-1], ensure_ascii=False)+'\n')
    idx +=1

100%|██████████| 1280/1280 [06:56<00:00,  3.08it/s]


In [16]:
dd=[]
raw_docs_long=[]
with open(saving_path, 'r', encoding='utf-8') as file:
    for f in file:
        temp =json.loads(f)
        dd.append(temp)
        testo=temp['text']
        title=temp['title']
        data_1=temp['data']
        argomento=temp['argomento']
        titolo_sporco=temp['fonte']
        raw_docs_long.append(Document(content=testo, meta={"testo_completo":testo, "title": title,"argomento":argomento,"data":str(data_1), "fonte":titolo_sporco}))

In [17]:
raw_docs_long[0]

Document(id=99b2baf6faa5c31366e804fccb42f4239fc3dc0b422100aa3aadd51acd7e8079, content: 'Certificati medici falsi e non attendibili: come contestare la giusta
causa di licenziamento.
Presen...', meta: {'testo_completo': 'Certificati medici falsi e non attendibili: come contestare la giusta\ncausa di licenziamento.\nPresentare certificati medici falsi per giustificare assenze dal lavoro a\ncausa della malattia di un figlio non giustifica un licenziamento, se il datore di\nlavoro non riesce a dimostrare che il lavoratore fosse consapevole della non\nautenticità di tali documenti.\nNon spetta al dipendente verificare l’attendibilità dei certificati medici; il suo\ncompito si limita solo alla trasmissione dei documenti al datore di lavoro.\nQuest’ultimo, se ne rileva la falsità, è tenuto a provare che il lavoratore fosse a\nconoscenza di tale non autenticità. Diversamente, il licenziamento per falsa\nmalattia del figlio risulta illegittimo.\nNon è possibile neanche presumere che l’uso di fa

In [18]:
len(raw_docs_long)

1266

In [24]:
len(raw_docs)

57312

In [49]:
raw_docs_long_greco=[]
with open('/home/a.alessandrelli/LLM/AvvoChat/Raccolta_doc_index/dataset_completo_13_07.jsonl', 'r', encoding='utf-8') as file:
    for f in file:
        temp=json.loads(f)
        testo=temp['corpo']
        title=temp['titolo']
        data_1=' '
        argomento=' '
        titolo_sporco=' '
        raw_docs_long_greco.append(Document(content=testo, meta={"testo_completo":testo, "title": title,"argomento":argomento,"data":str(data_1), "fonte":titolo_sporco}))

In [22]:
raw_docs=DocumentSplitter(split_by="sentence", split_length=5).run(raw_docs_long)['documents']

In [23]:
len(raw_docs)

8761

In [24]:
vett_text = [r.content for r in raw_docs]
vettorized_text = generate_embeddings(vett_text) 

***

In [25]:
import unicodedata

def rimuovi_caratteri_unicode(stringa):
    stringa_normalizzata = unicodedata.normalize('NFKD', stringa)
    stringa_ascii = stringa_normalizzata.encode('ascii', 'ignore').decode('ascii')
    return stringa.replace("\\\\","-").replace("\\","-").replace("/","-").replace("//","-")


raw_docs_vett = []
raw_docs_vett_saving = []
tot=0
for idx,d in enumerate(raw_docs):
    testo_originale=rimuovi_caratteri_unicode(str(raw_docs[idx].content))
    
    try:
        meta = {
                "id_originale" : raw_docs[idx].meta['source_id'],
                "text":testo_originale,
                "testo_completo" : raw_docs[idx].meta['testo_completo'],
                "titolo":raw_docs[idx].meta['title'],
                "argomento":raw_docs[idx].meta['argomento'],
                "data":raw_docs[idx].meta['data'],
                "anno":raw_docs[idx].meta['data'][0:4],
                'font':raw_docs[idx].meta['fonte'],
                }

        embeddings = vettorized_text[idx]
        raw_docs_vett.append({'text':meta, 'vector': embeddings})
        raw_docs_vett_saving.append({'text':meta})
    except:
        continue

In [26]:

# Funzione per costruire il dict del documento in base alla nuova struttura JSONL
def schema(d):
    dict ={
                "id_originale":d.get("id_originale",""),
                "text":d.get("text",""),
                "titolo":d.get("titolo",""),
                "testo_completo":d.get("testo_completo",""),
                "argomento":d.get("argomento",""),
                "anno":d.get("anno",""),
                "data":d.get("data",""),
                'font':d.get("font",""),
            }
    return dict


In [27]:

def class_schema(name_class):
    dict = {
        "class": name_class,
        "vectorizer": "none",
    }
    return dict



if client.schema.exists('Fisco'):
    tttt='ciao'
else:  
    class_cod = class_schema('Fisco')
    client.schema.create_class(class_cod)
    

# Caricamento e indicizzazione dei dati
print("\n\n Tot: ", len(raw_docs_vett), "\n")


client.batch.configure(batch_size=1500)  # Configure batch
with client.batch as batch:
    for i, d in enumerate(raw_docs_vett[34501:]):
        try:
            if (i % 1500 == 0):
                print(f"importing JSON block: {i + 1}")
            
            properties = schema(d['text'])
            batch.add_data_object(properties,class_name="Fisco", vector=d['vector'])
        except:
            continue
            
            
            
print("\n\nOK!!!!\n\n")



 Tot:  8761 



OK!!!!




In [37]:
dd_greco=[]
with open('/home/a.alessandrelli/LLM/AvvoChat/Raccolta_doc_index/codici_tot.jsonl', 'r', encoding='utf-8') as file:
    for f in file:
        dd_greco.append(json.loads(f))

In [38]:
dd_greco[0]

{'corpo': "Costituzione della Repubblica Italiana: IL CAPO PROVVISORIO DELLO STATO Vista la deliberazione dell'Assemblea Costituente, che nella seduta del 22 dicembre 1947 ha approvato la Costituzione della Repubblica Italiana; Vista la XVIII disposizione finale della Costituzione; PROMULGA la Costituzione della Repubblica Italiana nel seguente testo: Art. 1 L'Italia è una Repubblica democratica, fondata sul lavoro La sovranità appartiene al popolo, che la esercita nelle forme e nei limiti della Costituzione ",
 'titolo': 'Costituzione della Repubblica Italiana articolo 1'}